In [1]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn.apionly as sns
import datetime
import warnings
import pickle
from itertools import compress
warnings.filterwarnings('ignore')
% matplotlib inline

In [4]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge, Lasso
from sklearn.feature_selection import SelectFromModel

In [3]:
from model_generator_timeseries import *

### Load predictions from models

In [10]:
lgbm_df = pd.read_json('predictions/lgbm_predictions.json', orient='records')
rf_df = pd.read_json('predictions/rf_predictions.json', orient='records')
knn_df = pd.read_json('predictions/knn_predictions.json', orient='records')

In [12]:
lgbm_df = lgbm_df[['y_true', 'y_pred']].rename(columns={'y_pred':'lgbm', 'y_true':'band'})
rf_df = rf_df[['y_pred']].rename(columns={'y_pred':'rf'})
knn_df = knn_df[['y_pred']].rename(columns={'y_pred':'knn'})

stack_df = lgbm_df.merge(pd.concat([knn_df, rf_df], axis=1), how='left', left_index=True, right_index=True)